In [53]:
import pandas as pd
import numpy as np

In [54]:
df=pd.read_csv('Spotify_with_genre.csv')

In [55]:
df.head()

,artist_name,track_name,artist_pop,popularity,release_date,genre,track_uri,artist_uri
0,A Thousand Horses,Preachin' To The Choir,37.0,0.0,2017-06-02,Rap,6JKj9seJCshLVCfBLFp7dS,55RI2GNCfyXr0f14uIdhwd
1,Water Liars,You Work Days I Work Nights,29.0,10.0,2012,dnb,7L7u4PrNt5WMtVI9lHWlLm,6k3UpifDbb2ox25edM5j3P
2,Chambao,Sera,44.0,31.0,2007-11-06,Rap,12EyOqMPMyAKlKSynICAGN,2qhLqZ1pkiUl5HNrU2Nz0R
3,Jeezy,Big Shit Poppin,69.0,19.0,2007-08-07,Rap,3tCDjKseLQJhShfbeg4m7K,4yBK75WVCQXej1p04GWqxH
4,La Arrolladora Banda El Limón De Rene Camacho,No La Voy A Engañar,74.0,54.0,2012,Rap,0UICwSsSpUt98XkeCfupFn,5bSfBBCxY8QAk4Pifveisz


In [56]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.client import SpotifyException

In [57]:
client_id = '6444952d25014134affbddb6854f27c7'
client_secret = '49c9c42e4b54426785ae5856b45b719e'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [58]:
# Function to fetch track info
def get_track_info(track_name):
    results = sp.search(q=track_name, limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_uri = track['uri'].split(':')[-1]  # Extracting the track URI
        artist_name = track['artists'][0]['name']  # Extracting the artist name
        release_date = track['album']['release_date'][:4]  # Extracting the year of release
        popularity = track['popularity']  # Track popularity
        artist_pop = sp.artist(track['artists'][0]['id'])['popularity']  # Artist popularity
        return track_uri, artist_name, release_date, popularity, artist_pop
    else:
        return None, None, None, None, None

In [59]:
# Function to fetch specific audio features
def get_audio_features(track_url):
    audio_features = sp.audio_features([track_url])
    if audio_features:
        # Extracting only desired audio features
        selected_features = {
            'danceability': audio_features[0]['danceability'],
            'energy': audio_features[0]['energy'],
            'key': audio_features[0]['key'],
            'loudness': audio_features[0]['loudness'],
            'mode': audio_features[0]['mode'],
            'speechiness': audio_features[0]['speechiness'],
            'acousticness': audio_features[0]['acousticness'],
            'instrumentalness': audio_features[0]['instrumentalness'],
            'liveness': audio_features[0]['liveness'],
            'valence': audio_features[0]['valence'],
            'tempo': audio_features[0]['tempo']
        }
        return selected_features
    else:
        return None

In [61]:
# Example usage
track_name = input("Enter the track name: ")
track_uri, artist_name, release_date, popularity, artist_pop = get_track_info(track_name)
if track_uri:
    print("Track URL:", track_uri)
    print("Artist:", artist_name)
    print("Release Year:", release_date)
    print("Track Popularity:", popularity)
    print("Artist Popularity:", artist_pop)
    audio_features = get_audio_features(track_uri)
    if audio_features:
        print("Audio Features:", audio_features)
    else:
        print("Failed to fetch audio features.")
else:
    print("Track not found.")

Track URL: 3jJ5HgLRNydtLRghjWVaCG
Artist: Pritam
Release Year: 2013
Track Popularity: 65
Artist Popularity: 87
Audio Features: {'danceability': 0.611, 'energy': 0.612, 'key': 2, 'loudness': -7.978, 'mode': 1, 'speechiness': 0.114, 'acousticness': 0.441, 'instrumentalness': 0, 'liveness': 0.18, 'valence': 0.898, 'tempo': 171.841}


In [62]:
import pickle

In [63]:
# Load scaler and XGBoost model
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('xgb.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

c:\Users\Vitrag Khatadia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Vitrag Khatadia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [64]:
# Function to preprocess input features using scaler
def preprocess_input(audio_features):
    # Convert audio features to numpy array
    features_array = np.array(list(audio_features.values())).reshape(1, -1)
    # Scale the features
    scaled_features = scaler.transform(features_array)
    return scaled_features

In [65]:
# Function to make prediction using XGBoost model
def predict(features):
    return xgb_model.predict(features)[0]

In [66]:
# Preprocess input features
scaled_features = preprocess_input(audio_features)

c:\Users\Vitrag Khatadia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [67]:
# Make prediction
genre = predict(scaled_features)

In [74]:
# Create DataFrame
data_df = pd.DataFrame({
    "artist_pop": [artist_pop],
    "popularity": [popularity],
    "year": [release_date],
    "genre": [genre]
})

In [75]:
# Load the model
with open('birch.pkl', 'rb') as f:
    birch_model = pickle.load(f)

In [76]:
# Use the model to predict clusters
clusters = birch_model.predict(data_df)

In [77]:
clusters

array([1], dtype=int64)